In [ ]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Bidirectional, LSTM, Activation, Dropout, Embedding, Input
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import pandas as pd

import numpy as np
import json

In [ ]:
def import_log(filepath):
  df = pd.read_csv(filepath)
  return(df.values.tolist())

def remove_nan(lists):
  newlists = []
  for tr in lists:
    newlists.append([int(x) for x in tr if str(x) != 'nan'])
  return(newlists)

def number_to_one_hot_X(X, dict_size): #if we want 
  newX = []
  for example in X:
    new_ex = []
    for i in range(len(example)):
      onehot = [0]*dict_size #changed
      if example[i] != 0:
        onehot[example[i] - 1] = 1 #-1 because begin counting at 0
      new_ex.append(onehot)
    newX.append(new_ex)
  return(np.array(newX))

def create_XY_prefix(log, mappingsize, prefixlen):
  X = []
  Y = []
  for i in range(0, len(log)):
    for k in range(1, len(log[i])):
      X.append(log[i][max(0, k-prefixlen):k]) #get the prefix of 'encoded' activities
      y = [0] *(mappingsize)
      y[int(log[i][k])-1] = 1
      Y.append(y)        
  X = keras.preprocessing.sequence.pad_sequences(X, maxlen=prefixlen, padding='pre')
  X = number_to_one_hot_X(X, mappingsize)
  return(np.array(X), np.array(Y))



In [ ]:
def get_model(maxlen, num_chars, bidirec, n_layers, lstmsize, dropout, l1, l2):
  model = Sequential()
  model.add(Input(shape=(maxlen, num_chars))) #If you don't use an embedding layer input should be one-hot-encoded
  if bidirec == False:   
    model.add(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=(n_layers != 1),kernel_regularizer=regularizers.l1_l2(l1,l2),
                   recurrent_regularizer=regularizers.l1_l2(l1,l2),input_shape=(maxlen, num_chars)))
    model.add(Dropout(dropout))
    for i in range(1, n_layers):
      return_sequences = (i+1 != n_layers)
      model.add(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=return_sequences,
                     kernel_regularizer=regularizers.l1_l2(l1,l2),recurrent_regularizer=regularizers.l1_l2(l1,l2)))
      model.add(Dropout(dropout))
  else:
    model.add(Bidirectional(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=(n_layers != 1),kernel_regularizer=regularizers.l1_l2(l1,l2),
                   recurrent_regularizer=regularizers.l1_l2(l1,l2),input_shape=(maxlen, num_chars))))
    model.add(Dropout(dropout))
    for i in range(1, n_layers):
      return_sequences = (i+1 != n_layers)
      model.add(Bidirectional(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=return_sequences,
                     kernel_regularizer=regularizers.l1_l2(l1,l2),recurrent_regularizer=regularizers.l1_l2(l1,l2))))
      model.add(Dropout(dropout))
  model.add(Dense(num_chars, kernel_initializer='glorot_uniform',activation='softmax'))
  opt = Adam(learning_rate=0.005)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics='accuracy')
  return model


def do_one_experiment(X_train, y_train, X_test, y_test,batch_size, maxlen, num_chars, bidirec, n_layers, lstmsize, dropout, l1, l2):
  model = get_model(maxlen, num_chars, bidirec, n_layers, lstmsize, dropout, l1, l2)
  model.summary()
  early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
  #train_model
  history = model.fit(X_train, y_train, validation_split=0.2, callbacks=[early_stopping, lr_reducer], batch_size=batch_size, epochs=600, verbose=2)
  #test model on test set
  loss, accuracy = model.evaluate(X_test, y_test)
  return model, loss, accuracy

def do_grid_search(df, modelname, full_prefix, opt_prefixlen, training_log_location,location_mapping_activity_to_number, modelfilename):
  #add df just a way to continue training in seperate runs, keeps all results

  full_log = remove_nan(import_log(training_log_location)) #import full event log

  #if we want to use the full prefix each time or not
  if full_prefix == True:
    prefixlen=len(max(full_log,key=len)) -1
    print("prefix length:", prefixlen)
  else:
    prefixlen=opt_prefixlen

  #reload mapping
  mappingfilename = location_mapping_activity_to_number
  with open(mappingfilename) as f:
    mapping = json.loads(f.read())
  #split into train and test traces, the log is randomly played out, so no need to mix
  train_log = full_log[0: int(len(full_log) * 0.8)]
  test_log = full_log[int(len(full_log) * 0.8):-1]

  #convert into prefix + target, both one_hot_encoded
  X_train, y_train = create_XY_prefix(train_log, len(mapping), prefixlen)
  X_test, y_test = create_XY_prefix(test_log, len(mapping), prefixlen)
  
 

  grid_is_bidirectional = [False, True]
  grid_nr_layers = [1, 2]
  grid_layer_size = [16, 32, 64]
  grid_reg = [0.0, 0.0001, 0.001, 0.01]
  grid_dropout = [0.0 ,0.2, 0.4, 0.6]

  batch_size = 128

  i = 0 #keep track of index in grid

  for is_bidirectional in grid_is_bidirectional:
    for nr_layers in grid_nr_layers:
      for layer_size in grid_layer_size:
        for reg in grid_reg:
          for dropout in grid_dropout:
            if (i in df.index) == True:
              print(i, 'This setting already in dataframe:', is_bidirectional,nr_layers,layer_size,reg,dropout)
            else:
              model, loss, accuracy = do_one_experiment(X_train, y_train, X_test, y_test,batch_size, maxlen=prefixlen, num_chars=len(mapping), bidirec=is_bidirectional, n_layers=nr_layers, lstmsize=layer_size, dropout=dropout, l1=reg, l2=reg)
              new_row = {'Bidirectional':is_bidirectional,'Nr_layers':nr_layers,'Layer_size':layer_size, 'Regularization':reg,'Dropout':dropout,'Loss':loss,'Accuracy':accuracy}
              df = df.append(new_row, ignore_index=True)
              print(new_row)
              df.to_csv(path + 'Results_grid.csv')
              model.save(modelfilename)
            i = i+1